# Golam Saroare Shakil-011577991
# Assignment 4
# EconS 512

In [1]:
#Problem1
set.seed(100)
t<-runif(1000)  #Generating 1000 uniform random variable from [0,1]
intercept<-rep(1,n=length(t))  
x<-as.matrix(data.frame(cbind(intercept,t)))
u<-rnorm(1000)  #Generating 1000 Normmal random variable with mean 0 and standard deviation 1
beta<-c(5,2)    #My choice of parameters
y<-beta[1]+beta[2]*t+u
beta.est<-solve(t(x)%*%x)%*%(t(x)%*%y)  #Parameter estimate for Regression of Y on T 
beta.est

intercept,5.037536
t,1.985286


I generated 1000 uniform random variable from [0,1] as T 1000 standard normal variable as u. I choose $\beta_{0}=5$ and $\beta_{1}=2$. The OLS estimated coefficient for T is 1.985 which is close to the original parameter 2.

In [2]:
set.seed(100)
tau1<-t+rnorm(1000,0,0.2)   #Generating 1000 Normmal random variable with mean 0 and standard deviation 2
tau2<-t+rnorm(1000,0,0.3)  #Generating 1000 Normmal random variable with mean 0 and standard deviation 3
x1<-as.matrix(data.frame(cbind(intercept,tau1)))
beta.est1<-solve(t(x1)%*%x1)%*%(t(x1)%*%y)  #Parameter estimate for Regression of Y on Tau.1
beta.est_c<-beta[2]*(var(t)/(var(t)+0.04))
result<-data.frame(cbind(beta.est1[2],beta.est_c))
names(result)<-c("OLS estimate","Closed-form estimate")
result

OLS estimate,Closed-form estimate
1.309512,1.347162


Again, I generated 1000 normal random variable for each $\tau_{1}$ and $\tau_{2}$ with 0 mean and standard deviation 0.2 and 0.3 respectively. As we can see from the result, the OLS estimate is very close the closed form estimate. 

In [3]:
x2<-as.matrix(data.frame(cbind(intercept,tau2)))
iv.est<-solve(t(x2)%*%x1)%*%(t(x2)%*%y) #GMM estimate of IV
iv.est

intercept,5.047273
tau1,1.953816


Here, we used the GMM estimate of IV: $\beta_{IV}=(Z'X)^{-1}(Z'Y)$. We used $\tau_{2}$ as an instrument for $\tau_{1}$. The GMM estimate of $\beta_{1}$ is 1.95, which is close to our original parameter.

In [18]:
#Problem2
dat<-read.csv("midterm.csv",header=T)
names(dat)[1]<-"pubid"
dat<-na.omit(dat)
intrcpt<-rep(1,dim(dat)[1])
x<-as.matrix(cbind(intrcpt,pctinsclnxtyr=dat[,"pct_insclnxtyr"],mhighgrad=dat[,"mhighgrad"],msomcol=dat[,"msomcol"],
                   fhighgrad=dat[,"fhighgrad"],fsomcol=dat[,"fsomcol"],parincome=dat[,"parincome"],afqt=dat[,"afqt"]))
y<-dat[,"dayssmklm17"]
beta<-solve(t(x)%*%x)%*%(t(x)%*%y) #OLS estimation
res<-y-x%*%beta
sigma<-matrix(rep(0,dim(x)[1]*dim(x)[1]),nrow=dim(x)[1],ncol=dim(x)[1]) #Variance-Covariance structure: Assuming 
                                                                           #independence but non-identical 
for (i in 1:dim(x)[1]){
    for (j in 1:dim(x)[1]){
        if (i==j)sigma[i,j]<-res[i]^2
    }
}
covmat<-solve(t(x)%*%x)%*%t(x)%*%sigma%*%x%*%solve(t(x)%*%x) #Robust standard error estimation 
ci_up<-beta[2]+sqrt(covmat[2,2])*1.96
ci_low<-beta[2]-sqrt(covmat[2,2])*1.96
ci<-data.frame(cbind(ci_low,ci_up))     #Confidence interval for "pctinsclnxtyr"
names(ci)=c("Lower limit","Upper limit")
row.names(ci)<-"CI- OLS Est"
ci
beta

,Lower limit,Upper limit
CI- OLS Est,-0.08100386,-0.02249132


intrcpt,4.47802507
pctinsclnxtyr,-0.05174759
mhighgrad,1.70575780
msomcol,1.65431451
fhighgrad,2.04786694
fsomcol,-0.69472874
parincome,0.01560051
afqt,0.02546164


In [39]:
set.seed(100)
s<-1:dim(dat)[1]
l<-1000
beta.b<-matrix(rep(0,l*dim(x)[2]),nrow=l,ncol=dim(x)[2])
se.b<-c()
for(i in 1:l){
    rs<-sample(s,size=length(s),replace=T)
    beta.b[i,]<-solve(t(x[rs,])%*%x[rs,])%*%(t(x[rs,])%*%y[rs])
    resd<-y[rs]-x[rs,]%*%beta.b[i,]
    sig<-matrix(rep(0,dim(x[rs,])[1]*dim(x[rs,])[1]),nrow=dim(x[rs,])[1],ncol=dim(x[rs,])[1])
    for (k in 1:dim(x[rs,])[1]){
        for (j in 1:dim(x[rs,])[1]){
            if (k==j)sig[k,j]<-resd[k]^2
            }
    }
    cov<-solve(t(x[rs,])%*%x[rs,])%*%t(x[rs,])%*%sig%*%x[rs,]%*%solve(t(x[rs,])%*%x[rs,])
    se.b[i]<-sqrt(cov[2,2])
}
ci.bc<-quantile(beta.b[,2],c(0.025,0.975))#Boot-c confidence interval
ci.bc
t<-(beta.b[,2]-beta[2])/se.b
td<-quantile(t,c(0.025,0.975))
ci.bt_l<-beta[2]+sqrt(covmat[2,2])*td[1]
ci.bt_u<-beta[2]+sqrt(covmat[2,2])*td[2]
ci.bt<-data.frame(cbind(ci.bt_l,ci.bt_u))     #Boot-t confidence interval
names(ci.bt)=c("Lower limit","Upper limit")
ci.bt


2.5%       97.5% 
-0.08199123 -0.02217308

,Lower limit,Upper limit
2.5%,-0.07974,-0.01880163


In [30]:
z<-as.matrix(cbind(intrcpt,ctuition17=dat[,"ctuition17"],mhighgrad=dat[,"mhighgrad"],msomcol=dat[,"msomcol"],
                  fhighgrad=dat[,"fhighgrad"],fsomcol=dat[,"fsomcol"],parincome=dat[,"parincome"],afqt=dat[,"afqt"]))
beta_1st<-solve(t(z)%*%z)%*%(t(z)%*%dat[,"pct_insclnxtyr"]) #First stage of 2SLS 
pct.hat<-z%*%beta_1st
x.hat<-as.matrix(cbind(intrcpt,pct.hat,mhighgrad=dat[,"mhighgrad"],msomcol=dat[,"msomcol"],
                  fhighgrad=dat[,"fhighgrad"],fsomcol=dat[,"fsomcol"],parincome=dat[,"parincome"],afqt=dat[,"afqt"]))
beta_iv<-solve(t(x.hat)%*%x.hat)%*%(t(x.hat)%*%y)
rownames(beta_iv)[2]<-"pctinsclnxtyr"
res_iv<-y-x.hat%*%beta_iv
sigma_iv<-matrix(rep(0,dim(x.hat)[1]*dim(x.hat)[1]),nrow=dim(x.hat)[1],ncol=dim(x.hat)[1]) #Independent but non-identical 
                                                                                             #Variance-Covariance structure
for (i in 1:dim(x.hat)[1]){
    for (j in 1:dim(x.hat)[1]){
        if (i==j)sigma_iv[i,j]<-res_iv[i]^2
    }
}
covmat_iv<-solve(t(x.hat)%*%x.hat)%*%t(x.hat)%*%sigma_iv%*%x.hat%*%solve(t(x.hat)%*%x.hat) #Robust standard error estimation 
ci_up_iv<-beta_iv[2]+sqrt(covmat_iv[2,2])*1.96
ci_low_iv<-beta_iv[2]-sqrt(covmat_iv[2,2])*1.96
ci_iv<-data.frame(cbind(ci_low_iv,ci_up_iv))     #Confidence interval for IV estimate
names(ci_iv)=c("Lower limit","Upper limit")
row.names(ci_iv)<-"CI- IV Est"
ci_iv
beta_iv

,Lower limit,Upper limit
CI- IV Est,-0.3205731,0.09363099


intrcpt,8.38928057
pctinsclnxtyr,-0.11347104
mhighgrad,1.78092810
msomcol,1.96585034
fhighgrad,2.30796945
fsomcol,-0.19592583
parincome,0.01746634
afqt,0.03559152


In [35]:
set.seed(100)
s<-1:dim(dat)[1]
l<-1000
beta_iv.b<-matrix(rep(0,l*dim(x.hat)[2]),nrow=l,ncol=dim(x.hat)[2])
se_iv.b<-c()
for(i in 1:l){
    rs<-sample(s,size=length(s),replace=T)
    beta_1st<-solve(t(z[rs,])%*%z[rs,])%*%(t(z[rs,])%*%dat[,"pct_insclnxtyr"][rs])
    pct.hat<-z[rs,]%*%beta_1st
    x.hat<-as.matrix(cbind(intrcpt,pctinsclnxtyr=pct.hat,mhighgrad=dat[,"mhighgrad"],msomcol=dat[,"msomcol"],
                  fhighgrad=dat[,"fhighgrad"],fsomcol=dat[,"fsomcol"],parincome=dat[,"parincome"],afqt=dat[,"afqt"]))
    beta_iv.b[i,]<-solve(t(x.hat[rs,])%*%x.hat[rs,])%*%(t(x.hat[rs,])%*%y[rs])
    resd_iv<-y[rs]-x.hat[rs,]%*%beta_iv.b[i,]
    sig_iv<-matrix(rep(0,dim(x.hat[rs,])[1]*dim(x.hat[rs,])[1]),
                   nrow=dim(x.hat[rs,])[1],ncol=dim(x.hat[rs,])[1]) #Independent but non-identical 
                                                                    #Variance-Covariance structure
    for (k in 1:dim(x.hat[rs,])[1]){
        for (j in 1:dim(x.hat[rs,])[1]){
            if (k==j)sig_iv[k,j]<-resd_iv[k]^2
    }
}
cov_iv<-solve(t(x.hat[rs,])%*%x.hat[rs,])%*%t(x.hat[rs,])%*%sig_iv%*%x.hat[rs,]%*%solve(t(x.hat[rs,])%*%x.hat[rs,])
se_iv.b[i]<-sqrt(cov_iv[2,2])
}
ci.bc_iv<-quantile(beta_iv.b[,2],c(0.025,0.975))#Bootstrap-c Confidence interval for IV estimate
ci.bc_iv
t_iv<-(beta_iv.b[,2]-beta_iv[2])/se_iv.b
td_iv<-quantile(t_iv,c(0.025,0.975))
ci.bt_liv<-beta_iv[2]+sqrt(covmat_iv[2,2])*td_iv[1]
ci.bt_uiv<-beta_iv[2]+sqrt(covmat_iv[2,2])*td_iv[2]
ci.bt_iv<-data.frame(cbind(ci.bt_liv,ci.bt_uiv))     #Boot-t confidence interval
names(ci.bt_iv)=c("Lower limit","Upper limit")
ci.bt_iv

2.5%      97.5% 
-0.1300335  0.1205427

,Lower limit,Upper limit
2.5%,-0.1551784,0.5219737


From the four confidence interval, we can see that the ols estimate under heterogeneity with robust standard error estimate has $95\%$ confidence interval (CI) similar to that of what we get with bootstrap sampling. On the other hand, two stage least square (2SLS) produces a CI which is way off from the bootstrap estimate. These results goes along the point made in the recent paper of Alwyn Young where he asserted that, 2SLS is susciptible to non-iid relative to OLS. 